tout dabord , nous allons importer les données depuis le site de la banque mondiale à partir de leur API

In [2]:
print("welcome alex and elise. time to get start")
!pip install wbdata

welcome alex and elise. time to get start


In [ ]:
import wbdata
import pandas as pd
import datetime

# Définir les indicateurs que vous voulez récupérer
indicators = {
    'NY.GDP.MKTP.KD': 'GDP (constant 2015 US$)',
    'NE.GDI.FTOT.KD': 'Gross Fixed Capital Formation (constant 2015 US$)',
    'SP.POP.1564.TO': 'Population ages 15-64, total',
    'NE.GDI.FTOT.ZS': 'Gross Fixed Capital Formation (% GDP)'
}


# Télécharger les données
data = wbdata.get_dataframe(indicators, country= 'all')

# Afficher les premières lignes
print(data.head())

                                  GDP (constant 2015 US$)  \
country                     date                            
Africa Eastern and Southern 2023             1.068604e+12   

                                  Gross Fixed Capital Formation (constant 2015 US$)  \
country                     date                                                      
Africa Eastern and Southern 2023                                       2.892528e+11   

                                  Population ages 15-64, total  \
country                     date                                 
Africa Eastern and Southern 2023                   413786598.0   

                                  Gross Fixed Capital Formation (% GDP)  
country                     date                                         
Africa Eastern and Southern 2023                              19.792999  
